In [4]:
import PyPDF2

In [5]:
pdffileobj = open('ac1.pdf','rb')

In [6]:
pdfreader = PyPDF2.PdfFileReader(pdffileobj)

In [7]:
num_pages = pdfreader.numPages
print(num_pages)

69


In [8]:
text = ""
for page_number in range(30):
    pageobj = pdfreader.getPage(page_number)
    text += pageobj.extractText()

In [9]:
text = text.splitlines()

In [10]:
i = 6
print(text[i])
if text[i].strip():
    print(bool(text[i].strip()))
    print(text[i])

In [25]:
def cleaner(text_, idx_, cleaned_text_=[]):
    prv = idx_ - 1
    nxt = idx_ + 1 
    if nxt == len(text_):
        return cleaned_text_

    if not text_[nxt].strip() and not text_[prv].strip():
        cleaned_text_.append(text_[idx_])
        return cleaner(text_, nxt, cleaned_text_)
    elif text_[prv].strip() and text_[idx_].strip(): 
        text_[prv] = text_[prv]+text_[idx_]
        del text_[idx_]
        return cleaner(text_, idx_, cleaned_text_)
    else:
        return cleaner(text_, nxt, cleaned_text_)
    



In [26]:
cleaned_text = cleaner(text, 1)

In [28]:
from gtts import gTTS
import genanki
from uuid import uuid4

In [ ]:
deck = genanki.Deck(22329839281, 'Deutsch Englisch (C1)')
model = genanki.Model(
    787837837,
    'Simple Model',
    fields=[
        {'name': 'Deutsch'},
        {'name': 'Englisch'},
    ],
    templates=[
        {
            'name': 'Card1',
            'qfmt': '{{Deutsch}}',
            'afmt': '{{FrontSide}}<hr id="answer">{{Deutsch}}'
        },
    ])
package = genanki.Package(deck)
mediafiles = []

for idx, t in enumerate(cleaned_text):
    if not bool(t.strip()):
        continue
    print(t)
    guid = str(uuid4())
    print(guid)
    tts =  gTTS(text=t, lang='de', slow=False)
    with open('{}.mp3'.format(guid),'wb') as f:
        tts.write_to_fp(f)
    mediafiles.append('{}.mp3'.format(guid))
    my_note = genanki.Note(
        model=model,
        fields=['{}, [sound:{}.mp3]'.format(t, guid), t]
    )
    deck.add_note(my_note)

package.media_files = mediafiles
package.write_to_file('out_assimil.apkg')

